In [9]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime

def extrair_dados_financeiros(ticker_symbol, start_date, end_date):
    ticker = yf.Ticker(ticker_symbol)
    
    dados = {
        'Preço': yf.download(tickers=ticker_symbol, start=start_date, end=end_date),
        'Info': ticker.info,
        'Dividendos': ticker.dividends,
        'Splits': ticker.splits,
        'Financeiros': {
            'Balanço': ticker.balance_sheet,
            'DRE': ticker.financials,
            'Fluxo_Caixa': ticker.cashflow
        },
        'Sustentabilidade': ticker.sustainability,
        'Recomendações': ticker.recommendations,
        'Calendário': ticker.calendar
    }
    
    return dados

def calcular_indicadores_completos(dados):
    
    indicadores = pd.DataFrame()

    if 'DRE' in dados['Financeiros']:
        dre = dados['Financeiros']['DRE']
        if not dre.empty:
            indicadores['LPA'] = dre.loc['Net Income'] / dados['Info'].get('sharesOutstanding', 1)
            
            indicadores['Margem_Liquida'] = dre.loc['Net Income'] / dre.loc['Total Revenue']
            
            if 'EBITDA' in dre.index:
                indicadores['Margem_EBITDA'] = dre.loc['EBITDA'] / dre.loc['Total Revenue']
    
    if not dados['Dividendos'].empty:
        indicadores['Dividend_Yield'] = dados['Dividendos'].groupby(dados['Dividendos'].index.year).sum() / dados['Preço']['Close'].groupby(dados['Preço'].index.year).mean()
    
    return indicadores

ticker_symbol = "BEEF3.SA"
start_date = "2020-01-01"
end_date = "2023-12-31"

dados = extrair_dados_financeiros(ticker_symbol, start_date, end_date)

indicadores = calcular_indicadores_completos(dados)

print("\nIndicadores de Mercado:")
print(indicadores)

print("\nInformações Disponíveis:")
for key in dados.keys():
    print(f"- {key}")

[*********************100%***********************]  1 of 1 completed
404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/BEEF3.SA?modules=esgScores&corsDomain=finance.yahoo.com&formatted=false&symbol=BEEF3.SA&crumb=jHBCmMQfJzg


ValueError: Cannot set a DataFrame with multiple columns to the single column Dividend_Yield

In [ ]:
def calcular_patrimonio_liquido_medio(dados):
    balanco = dados['Financeiros']['Balanço']
    
    patrimonio_liquido = balanco.loc['Total Equity Gross Minority Interest']
    
    pl_medio = pd.Series(index=patrimonio_liquido.index)
    
    for i in range(len(patrimonio_liquido)):
        if i == 0: 
            pl_medio.iloc[i] = patrimonio_liquido.iloc[1] 
        else:
            pl_medio.iloc[i] = patrimonio_liquido.iloc[i:i+2].mean()
    
    print("Patrimônio Líquido:")
    print(patrimonio_liquido)
    print("\nPatrimônio Líquido Médio (ajustado):")
    print(pl_medio)
    
    return pl_medio

pl_medio = calcular_patrimonio_liquido_medio(dados)

Patrimônio Líquido:
2023-12-31    78975000000.0
2022-12-31    69836000000.0
2021-12-31    69812000000.0
2020-12-31    59876000000.0
Name: Total Equity Gross Minority Interest, dtype: object

Patrimônio Líquido Médio (ajustado):
2023-12-31    6.983600e+10
2022-12-31    6.982400e+10
2021-12-31    6.484400e+10
2020-12-31    5.987600e+10
dtype: float64


In [ ]:
def calcular_indicadores_fundamentalistas(dados):
    lucro_liquido = dados['Financeiros']['DRE'].loc['Net Income']
    patrimonio_liquido = dados['Financeiros']['Balanço'].loc['Total Equity Gross Minority Interest']
    total_ativos = dados['Financeiros']['Balanço'].loc['Total Assets']
    num_acoes = dados['Financeiros']['Balanço'].loc['Ordinary Shares Number']
    receita_total = dados['Financeiros']['DRE'].loc['Total Revenue']
    precos_diarios = dados['Preço']['Adj Close']
    
    pl_medio = pd.Series(index=patrimonio_liquido.index)
    for i in range(len(patrimonio_liquido)):
        if i == 0:
            pl_medio.iloc[i] = patrimonio_liquido.iloc[1]
        else:
            pl_medio.iloc[i] = patrimonio_liquido.iloc[i:i+2].mean()
    
    indicadores = pd.DataFrame(index=lucro_liquido.index)
    
    indicadores['ROE'] = lucro_liquido / pl_medio
    indicadores['ROA'] = lucro_liquido / total_ativos
    indicadores['VP'] = patrimonio_liquido / num_acoes
    
    indicadores['ML'] = lucro_liquido / receita_total  
    
    precos_anuais = precos_diarios.groupby(precos_diarios.index.year).last()
    
    precos_anuais.index = pd.to_datetime(precos_anuais.index.astype(str) + '-12-31')
    precos_anuais = precos_anuais.reindex(lucro_liquido.index)
    
    print("Preços Anuais (último dia do ano):")
    print(precos_anuais)
    
    lpa = lucro_liquido / num_acoes  
    indicadores['P/L'] = precos_anuais / lpa 
    indicadores['P/VPA'] = precos_anuais / indicadores['VP'] 
    
    return indicadores

def criar_matriz_correlacao_diaria(dados, indicadores_fundamentalistas):
    precos_diarios = dados['Preço']['Adj Close']
    
    indicadores_diarios = pd.DataFrame(index=precos_diarios.index)
    
    for coluna in ['ROE', 'ROA', 'VP', 'ML', 'P/L', 'P/VPA']:
        indicadores_diarios[coluna] = indicadores_fundamentalistas[coluna].reindex(
            precos_diarios.index, method='ffill'
        )
    
    dados_combinados = pd.DataFrame({
        'Preço': precos_diarios,
        **{col: indicadores_diarios[col] for col in indicadores_diarios.columns}
    })
    
    matriz_correlacao = dados_combinados.corr()
    
    return dados_combinados, matriz_correlacao

indicadores_fundamentalistas = calcular_indicadores_fundamentalistas(dados)
print("\nIndicadores Fundamentalistas:")
print(indicadores_fundamentalistas)

Preços Anuais (último dia do ano):
2023-12-31    30.414377
2022-12-31    15.453549
2021-12-31    11.227673
2020-12-31     9.090478
Name: Adj Close, dtype: float64

Indicadores Fundamentalistas:
                 ROE       ROA        VP        ML         P/L     P/VPA
2023-12-31  0.356321  0.114637  6.103104  0.242986   15.816035  4.983428
2022-12-31  0.524504  0.195645  5.353797  0.294222     5.50417  2.886466
2021-12-31  0.306505  0.113996  5.351957  0.236703    7.368856  2.097863
2020-12-31  0.019056  0.006005  4.590239  0.021254  103.924648  1.980393


In [ ]:
def criar_matriz_correlacao_diaria(dados, indicadores_fundamentalistas):
    precos_diarios = dados['Preço']['Adj Close']
    
    indicadores_diarios = pd.DataFrame(index=precos_diarios.index)
    
    for coluna in ['ROE', 'ROA', 'VP', 'ML', 'P/L', 'P/VPA']:
        indicadores_diarios[coluna] = indicadores_fundamentalistas[coluna].reindex(
            precos_diarios.index, method='ffill'
        )
    
    dados_combinados = pd.DataFrame({
        'Preço': precos_diarios,
        'ROE': indicadores_diarios['ROE'],
        'ROA': indicadores_diarios['ROA'],
        'VP': indicadores_diarios['VP'],
        'ML': indicadores_diarios['ML'],
        'P/L': indicadores_diarios['P/L'],
        'P/VPA': indicadores_diarios['P/VPA']
    })

    dados_combinados['Preço'] = dados_combinados['Preço'].shift(-1)

    dados_combinados.dropna(inplace=True)
    
    matriz_correlacao = dados_combinados.corr()
    
    import seaborn as sns
    import matplotlib.pyplot as plt
    
    plt.figure(figsize=(12, 10))
    sns.heatmap(matriz_correlacao, 
                annot=True,
                fmt='.2f',
                cmap='coolwarm',
                vmin=-1, 
                vmax=1,
                center=0)
    plt.title(f'Matriz de Correlação Diária: Preço (dia seguinte) vs Indicadores Microeconômicos ({ticker_symbol})')
    plt.xticks(rotation=45)
    plt.yticks(rotation=45)
    plt.tight_layout()
    plt.show()
    
    return dados_combinados, matriz_correlacao

dados_diarios, matriz_correlacao = criar_matriz_correlacao_diaria(dados, indicadores_fundamentalistas)
print("\nMatriz de Correlação:")
print(matriz_correlacao)

NameError: name 'indicadores_fundamentalistas' is not defined